## Goal: We want to know to what novel exons appear because they have at least one highly-divegent variant position in them 
We'll limit this analyses to the contrasts that have CEU in them, because we know that the annotation is European-biased, and therefore the novelty of exons is dependent on this bias

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot
import math


p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [3]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [4]:
df = pd.read_csv(proc_cfg(config['lr']['exons_ss_in_pops_fsts_summary'], od),
                 sep='\t')
df.rename({'pop2_der':'pop2_det'}, 
          axis=1, inplace=True)

# set all <0 fsts --> 0 https://www.biostars.org/p/132253/
df.loc[df.fst<0, 'fst'] = 0

# since this is biallelic snp level, remove all exons that didnt intersect
print(len(df.index))
df = df.loc[df.End_b!=-1]
print(len(df.index))

2943235
1719222


In [5]:
## For each exon / contrast, get the max fst
df = df.sort_values(by=
df = df.drop_duplicates(subset=['eid', 'pop1', 'pop2'])

,eid,novelty,pop1_det,pop2_det,Chromosome,Strand,Start,End,pop1,pop2,End_b,fst,Start_b
197,chr1_+_100007033_100007156,Known,True,True,chr1,+,100007031,100007158,HAC,YRI,100007048,0.393014,100007047
198,chr1_+_100043072_100043229,Known,True,True,chr1,+,100043070,100043231,HAC,YRI,100043099,0.041198,100043098
199,chr1_+_100043072_100043229,Known,True,True,chr1,+,100043070,100043231,HAC,YRI,100043213,0.000000,100043212
200,chr1_+_100049908_100050004,Known,True,True,chr1,+,100049906,100050006,HAC,YRI,100049941,0.018298,100049940
201,chr1_+_100140407_100141019,Known,True,True,chr1,+,100140405,100141021,HAC,YRI,100140797,0.047965,100140796
